In [1]:

from openai import OpenAI

client = OpenAI()

META_PROMPT = """
Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
""".strip()

def generate_prompt(task_or_prompt: str):
  completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
          {
              "role": "system",
              "content": META_PROMPT,
          },
          {
              "role": "user",
              "content": "Task, Goal, or Current Prompt:\n" + task_or_prompt,
          },
      ],
  )

  return completion.choices[0].message.content

In [2]:

META_PROMPT2 = """
Given a current prompt and a change description, produce a detailed system prompt to guide a language model in completing the task effectively.

Your final output will be the full corrected prompt verbatim. However, before that, at the very beginning of your response, use <reasoning> tags to analyze the prompt and determine the following, explicitly:
<reasoning>
- Simple Change: (yes/no) Is the change description explicit and simple? (If so, skip the rest of these questions.)
- Reasoning: (yes/no) Does the current prompt use reasoning, analysis, or chain of thought? 
    - Identify: (max 10 words) if so, which section(s) utilize reasoning?
    - Conclusion: (yes/no) is the chain of thought used to determine a conclusion?
    - Ordering: (before/after) is the chain of though located before or after 
- Structure: (yes/no) does the input prompt have a well defined structure
- Examples: (yes/no) does the input prompt have few-shot examples
    - Representative: (1-5) if present, how representative are the examples?
- Complexity: (1-5) how complex is the input prompt?
    - Task: (1-5) how complex is the implied task?
    - Necessity: ()
- Specificity: (1-5) how detailed and specific is the prompt? (not to be confused with length)
- Prioritization: (list) what 1-3 categories are the MOST important to address.
- Conclusion: (max 30 words) given the previous assessment, give a very concise, imperative description of what should be changed and how. this does not have to adhere strictly to only the categories listed
</reasoning>
    
# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
[NOTE: you must start with a <reasoning> section. the immediate next token you produce should be <reasoning>]
""".strip()

def generate_prompt2(task_or_prompt: str):
  completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
          {
              "role": "system",
              "content": META_PROMPT2,
          },
          {
              "role": "user",
              "content": "Task, Goal, or Current Prompt:\n" + task_or_prompt,
          },
      ],
  )

  return completion.choices[0].message.content

In [3]:
temp_prompt = """
### RISEN Framework for Horoscope Interpretation

**Role (R):**
Astrology Expert

**Instructions (I):**
Interpret the houses and planets in the given horoscope image and explain the significance of each.
Here is the legend:
- Lag: Lagna (1st house or Ascendant)
- Su: Sun
- Mo: Moon
- Me: Mercury
- Ve: Venus
- Ma: Mars
- Ju: Jupiter
- Sa: Saturn
- Ra: Rahu (North Node of the Moon)
- Ke: Ketu (South Node of the Moon)

**End Goal (E):**
**Steps (S):**
1. Identify the Lagna (Ascendant) and its corresponding house.
2. Locate each planet (Sun, Moon, Mercury, Venus, Mars, Jupiter, Saturn, Rahu, Ketu) in the horoscope and the box nuber which it is in.
3. Box number represents the house number.
4. Interpret the significance of each house and the planet residing in it.
5. Summarize the overall personality based on the planetary positions and house influences.

Provide a detailed interpretation of the horoscope, highlighting the significance of each house and planet, and summarize the personality traits of the individual.

**Narrowing (N):**
- Do not explain the image or any other concepts of astrology.
- Focus solely on interpreting the houses and planets.
- Use the provided legend for planet abbreviations.
- Keep the explanation concise and to the point.
- Create response in nice structured format with proper headings and bullet points for easy reading and understanding.
- Use not more than 1000 words for the entire interpretation.
""".strip()

# """You are parashara maharshi, a great astrologer. You are given complete birth chart details.
# your task is to answer the most recent question based on the birth chart details. """

generated_prompt = generate_prompt(temp_prompt)



In [4]:
print(generated_prompt)

Create separate files for each heading section in the provided README and fill in the data for each subsection within those chapters.

- Begin by dividing the README into sections and subsections based on headings.
- Address each subsection sequentially, ensuring a comprehensive review from start to finish.
- Use web search resources such as Perplexity-Ask or Brave Search for additional information when necessary.
- Extract insights such as entities, relationships, and potential knowledge graphs from the content of each subsection, keeping a record for each section.

# Steps

1. **Divide the README:**
   - Identify all the main and subheadings within the README.
   - Create separate files for each main heading section.

2. **Fill in Data:**
   - For each subsection within a section, review the content and start adding information to the respective files.
   - Conduct web searches if needed to enhance the content or fill in missing information.

3. **Insight Extraction:**
   - Extract a

In [20]:
temp_prompt = """With all these notes: create a linkedin post which explains the current situation and what a great time to be part of the industry and witness this. add more emojies and make is grasping and fun for readers yet keeping it nerdy and professional:
divide the post into below sections and rephrase the headding:
what happened so far,
a little technology behind it and why is it a big deal.
what is happening now as a result.
current state of deepseek and president callout
where the future is heading. keep the overall instruction less than 1500 characters.
""".strip()

generated_prompt = generate_prompt(temp_prompt)

print(generated_prompt)

Create a LinkedIn post that explains the current state of the industry and highlights why it is an exciting time to be part of it. Use emojis to enhance engagement while maintaining a nerdy and professional tone. Divide the post into sections with rephrased headings as follows:

# Overview of Recent Developments

Summarize significant events and changes that have recently occurred in the industry.

# Tech Insights: Behind the Scenes

Briefly explain the technology driving these changes and why they are significant.

# Today's Landscape

Describe what is currently happening in the industry as a result of recent developments.

# Spotlight on DeepSeek and Leadership Recognition

Highlight DeepSeek's current status and include a mention of leadership or presidential acknowledgment.

# Future Trajectory

Discuss the potential directions the industry might take moving forward.

# Output Format

The post should be less than 1500 characters with professional and engaging language. Use emojis s

In [19]:
generated_prompt2 = generate_prompt2(generated_prompt+" reduce the length of the final instruction to less than 2000 characters")

print(generated_prompt2)


<reasoning>
- Simple Change: yes
- Reasoning: no
    - Identify:
    - Conclusion: 
    - Ordering: 
- Structure: yes
- Examples: yes
    - Representative: 5
- Complexity: 1
    - Task: 1
    - Necessity:
- Specificity: 5
- Prioritization: Overall length, Engaging tone, Use of emojis
- Conclusion: Simplify the guidelines to ensure LinkedIn character limits and reduce redundancy in the task description.
</reasoning>

Create a LinkedIn post that explains the current situation in your industry and highlights why it's an exciting time to be part of it. The post should be engaging, with a nerdy and professional tone, and include emojis to make it fun for readers. Divide the post into the sections outlined below, keeping the overall length under 2000 characters.

# Sections

### What's the Buzz So Far?
Summarize the major developments or events that have recently occurred in the industry in an engaging manner.

### The Tech Behind the Magic ✨
Provide a brief explanation of the technology inv

In [3]:
# Please install OpenAI SDK first: `pip3 install openai`
import os
from openai import OpenAI
api_key = "sk-051420bb66b7495ca91c920ce184ae6f"
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

JSONDecodeError: Expecting value: line 5 column 1 (char 4)

In [1]:
from openai import OpenAI
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

# Round 1
messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    stream=True
)

reasoning_content = ""
content = ""
for chunk in response:
    if hasattr(chunk.choices[0], 'delta'):
        if hasattr(chunk.choices[0].delta, 'content'):
            content += chunk.choices[0].delta.content
            print(chunk.choices[0].delta.content, end='')
        # If 'reasoning_content' is expected, check if it exists
        elif hasattr(chunk.choices[0].delta, 'reasoning_content'):
            reasoning_content += chunk.choices[0].delta.reasoning_content
            print(chunk.choices[0].delta.reasoning_content, end='')
    else:
        content += chunk.choices[0].delta.content
        print(chunk.choices[0].delta.content, end='')

# Round 2
messages.append({"role": "assistant", "content": content})
messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)
# ...

NameError: name 'api_key' is not defined

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
api_key = os.getenv("HUGGINGFACE_API_KEY")
# api_key="" #check env
base_url = os.getenv("HUGGINGFACE_API_BASE_URL")
model = os.getenv("HUGGINGFACE_MODEL")

print(api_key, base_url, model)
client = OpenAI(
	base_url=base_url,
	api_key=api_key
)

messages = [
	{
		"role": "user",
		"content": "What is the capital of France?"
	}
]

completion = client.chat.completions.create(
	model=model, 
	messages=messages, 
	max_tokens=500
)

print(completion.choices[0].message)

None None None


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
curl 'https://huggingface.co/api/inference-proxy/together/v1/chat/completions' \
-H 'Authorization: Bearer <check_env>' \
-H 'Content-Type: application/json' \
--data '{
    "model": "deepseek-ai/DeepSeek-V3",
    "messages": [
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
    "max_tokens": 500,
    "stream": false
}'


In [ ]:
print(generated_prompt)

In [10]:
temp_prompt = """

Create a personal blog post summarizing data from a YouTube transcript. The post should be conversational, engaging, and not self-referential, summarizing key takeaways without quoting speakers directly. 

- Ensure the content is suitable for platforms like LinkedIn, Medium, X, or Facebook.
- Strive to make the post both fun and professional, with a nerdy twist.
- Structure the post with clear and concise sections.

# Steps

1. **Divide Content into Sections:**
   - Identify main topics or themes in the transcript.
   - For each theme, create a section starting with an appropriate emoji.
   - Use bullet points to highlight key ideas or insights within each section.

2. **Keep it Engaging:**
   - Maintain a conversational tone.
   - Use language that encourages comments and shares.

3. **Length and Clarity:**
   - Ensure the total post length is between 200 words.
   - Focus on the essence and main message without overly detailed descriptions.

4. **Add Hashtags:**
   - Include relevant hashtags at the end to enhance reach and engagement.

# Output Format

The output should be a short blog post of 200 words, structured with sections starting with emojis, and concluded with relevant hashtags. Each section should have bullet points to highlight information clearly.
""".strip()

# """You are parashara maharshi, a great astrologer. You are given complete birth chart details.
# your task is to answer the most recent question based on the birth chart details. """

generated_prompt = generate_prompt(temp_prompt)



In [11]:
print(generated_prompt)

You are Parashara Maharshi, a great astrologer. Your task is to answer questions based on the provided birth chart details with deep insight and astrological reasoning. Take into account all aspects of the birth chart, including planetary positions, houses, aspects, and other relevant factors to provide a comprehensive analysis before arriving at a conclusion.

# Steps 

1. **Analyze the Birth Chart:**
   - Examine the positions of all planets and their aspects.
   - Consider the role of houses and zodiac signs.
   - Evaluate any significant astrological phenomena that may influence the chart.

2. **Interpretation:**
   - Identify key patterns or notable configurations in the chart.
   - Use astrological principles and your extensive knowledge to draw insights.

3. **Address the Question:**
   - Relate your interpretations to the specific question asked.
   - Provide a well-reasoned analysis that references astrological wisdom.

4. **Remedies:**
   - Suggest remedies for any identified

In [7]:
temp_prompt = """

Create a personal blog post summarizing data from a YouTube transcript. The post should be conversational, engaging, and not self-referential, summarizing key takeaways without quoting speakers directly. 
- if author is provided, quote the author in the post.
- the context is given as a json object.
- each json object represent a specific session or content which has title, author, and transcript fields with other optional fields.
- Start with summarizing the content of each session, with providing the title of the session, and the author of the session.
- then proceed to write a blog post following the steps below.
- Strive to make the post both fun and professional, with a nerdy twist.
- Structure the post with clear and concise sections.

# Steps

1. **Divide Content into Sections:**
   - Identify main topics or themes in the transcript.
   - For each theme, create a section starting with an appropriate emoji.
   - Use bullet points to highlight key ideas or insights within each section.

2. **Keep it Engaging:**
   - Maintain a conversational tone.
   - Use language that encourages comments and shares.

3. **Length and Clarity:**
   - Ensure the total post length is between less than 500 words.
   - Focus on the essence and main message without overly detailed descriptions.

4. **Add Hashtags:**
   - Include relevant hashtags at the end to enhance reach and engagement.

dont give any examples.
# Output Format

The output should be a blog post of less than 500 words, structured with sections starting with emojis, and concluded with relevant hashtags. Each section should have bullet points to highlight information clearly.
""".strip()

generated_prompt = generate_prompt(temp_prompt)
print(generated_prompt)


Create a personal blog post summarizing data from a YouTube transcript using a provided context in JSON format. The blog post should be engaging, conversational, with a nerdy twist, and not directly quote speakers. If an author is provided, quote them in the blog post.

Each JSON object represents a session or content with fields like title, author, and transcript, among other optional fields. 

# Steps

1. **Summarize Content:**
   - Start by summarizing the content of each session.
   - Include the title and author of each session.

2. **Divide Content into Sections:**
   - Identify the main topics or themes from the transcript.
   - Create sections for each theme, starting with an appropriate emoji.
   - Use bullet points to highlight key ideas or insights.

3. **Craft an Engaging Tone:**
   - Use a conversational tone to maintain reader interest.
   - Opt for language that encourages reader interaction, such as comments and shares.

4. **Maintain Length and Clarity:**
   - Keep the